In [2]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)

In [2]:
datas= 'data_samples/'
# vocabulary = 'vocab.csv'
# vocabs = []
# with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
#     words = csv.reader(vocab)
#     for word in words:
#         vocabs.append(word)
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
# word_to_id
# inv_map

FileNotFoundError: [Errno 2] No such file or directory: 'data_samples/mapping.map'

In [3]:
def attention(inputs):
    
    # inputs = Batch * hidden
    inputs_shape = inputs.shape
#     sequence_length = inputs_shape[1].value
    hidden_size = inputs_shape[1].value
    
    w_omega = tf.Variable(tf.random_normal([hidden_size,attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    
    # batch *k , attention_size
    v = tf.tanh(tf.matmul(tf.reshape(inputs, [-1,hidden_size]), w_omega)+
               tf.reshape(b_omega,[1,-1]))
    
    vu = tf.matmul(v, tf.reshape(u_omega,[-1,1])) # batch*k,1
    
    exps = tf.reshape(tf.exp(vu), [-1, sequence])  # (batch, k)
    alphas = exps / tf.reshape(tf.reduce_sum(exps,1), [-1,1])
    #(batch,k)
    output = tf.reduce_sum(inputs * tf.reshape(alphas, [-1, sequence,1]),1)
    # batch * sequence * hidden -> batch * hidden
    return output

In [4]:
def make_attention(inputs, seq_len, initial_state):
    
    state = initial_state
    outputs = []
    states = []
    with tf.variable_scope("RNN"):
        time = tf.constant(0)
        for time_step in  range(seq_len):
            if time_step>0:
                tf.get_variable_scope().reuse_variables()
            print('time_step', inputs[:,time_step,:])
            output = attention(inputs[:,time_step,:])
            outputs.append(output)
            states.append(state)
            
        output_tensor = tf.stack(outputs)
        #output_tensor : time_step * 
        
    return output_tensor, state

In [5]:
hidden_size=32
sequence=5
embedding_dim=50
attention_size = 50
batch_size=16
vocab_size=len(word_to_id)
grucell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
# Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
Y = tf.placeholder(tf.int32, [None, sequence], name='targets_ydata')
seq_len = tf.placeholder(tf.int32, [None], name='seq_len')

embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
# inputs_ = tf.transpose(X)
batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)

print(batch_embedded)
with tf.variable_scope('RNN'):
    outputs, states = tf.nn.dynamic_rnn(grucell,
                                        inputs=batch_embedded,
                                        sequence_length=seq_len,
                                        dtype=tf.float32)

Tensor("embedding_lookup:0", shape=(?, 5, 50), dtype=float32)


In [22]:
initial_state = grucell.zero_state(batch_size, tf.float32)

In [27]:
print(outputs[:,3,:])
make_attention(outputs, sequence, initial_state)

Tensor("strided_slice_10:0", shape=(?, 32), dtype=float32)
time_step Tensor("RNN_15/strided_slice:0", shape=(?, 32), dtype=float32)
time_step Tensor("RNN_15/strided_slice_2:0", shape=(?, 32), dtype=float32)
time_step Tensor("RNN_15/strided_slice_4:0", shape=(?, 32), dtype=float32)
time_step Tensor("RNN_15/strided_slice_6:0", shape=(?, 32), dtype=float32)
time_step Tensor("RNN_15/strided_slice_8:0", shape=(?, 32), dtype=float32)


(<tf.Tensor 'RNN_15/stack:0' shape=(5, ?, 32) dtype=float32>,
 LSTMStateTuple(c=<tf.Tensor 'BasicLSTMCellZeroState_1/zeros:0' shape=(16, 32) dtype=float32>, h=<tf.Tensor 'BasicLSTMCellZeroState_1/zeros_1:0' shape=(16, 32) dtype=float32>))

In [7]:
attention_output = attention(outputs, attention_size=attention_size)
attention_output

<tf.Tensor 'Sum_1:0' shape=(?, 32) dtype=float32>

In [12]:
Y = tf.reshape(Y, [batch_size*sequence, 1])

In [13]:
keep_prob = 0.5
drop = tf.nn.dropout(attention_output, keep_prob=keep_prob)
w = tf.Variable(tf.truncated_normal([hidden_size,vocab_size], stddev=0.1))
b = tf.Variable(tf.constant(0., shape=[vocab_size]))
y_hat = tf.nn.xw_plus_b(drop,w,b)

# y_hat = tf.squeeze(y_hat)

# loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#     logits=y_hat,
#     labels=Y))
# loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=Y)
labels = tf.reshape(Y, [-1], name='label_reshape')
# print(y_hat, Y, labels, tf.transpose(w))
print(vocab_size)

loss = tf.nn.sampled_softmax_loss(weights=tf.transpose(w), biases=b, labels=Y,
                                  inputs=attention_output, num_sampled = 100, num_classes=vocab_size
                                 )

optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

14760


### accuracy 구하기

In [17]:
# accuracy = 1. - tf.reduce_mean(tf.cast(tf.equal(tf.round(y_hat), Y), tf.float32))

### make Batch

In [18]:
file_name_queue = tf.train.string_input_producer(['inputs'])
reader = tf.TFRecordReader()
_, single_x = reader.read(file_name_queue)

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}


context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                 context_features=context_features,
                                 sequence_features=sequence_features
                                )
batch_seq, batch_x, batch_y = tf.train.batch(tensors=[context_parsed['length'],sequence_parsed['tokens'],sequence_parsed['labels']],
                                                     batch_size=batch_size, 
                                                     capacity=5000,
                                                     num_threads=1,
                                             dynamic_pad=True
                                                    )

In [26]:
num_epochs = 10
delta = 0.5

Ses = tf.Session()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())

print('start learning!')

for epoch in range(num_epochs):
    
    print('{} start'.format(epoch))
    
    loss_train = 0
    loss_test = 0
    accuracy_train = 0
    accuracy_test =0
    
    
#     for i in range(1000):
    i=0
#     try:
    while True:
        seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
#         print('y_in', np.reshape(y_in,[-1,1]))
        y_in = np.reshape(y_in,[-1,1])
        feed_data = {X:x_in, Y:y_in, seq_len:seq_}
        print('x_in.shape, seq.shape', x_in.shape, seq_.shape)
#         lo, acc, opt = Ses.run([loss, accuracy, optimizer], feed_dict=feed_data)
        lo, opt = Ses.run([loss, optimizer], feed_dict=feed_data)
#         accuracy_train+=acc
        loss_train = lo * delta + loss_train * (1-delta)
        if i%100 ==0:
            print(' {} : loss {} acc {}'.format(i, lo, accuracy_train))
#     except:
#         print('batch : ',i)
#         accuracy_train /=

start learning!
0 start
x_in.shape, seq.shape (16, 5) (16,)


InvalidArgumentError: Incompatible shapes: [16,1,32] vs. [80,1,32]
	 [[Node: gradients/sampled_softmax_loss_2/Mul_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](gradients/sampled_softmax_loss_2/Mul_grad/Shape, gradients/sampled_softmax_loss_2/Mul_grad/Shape_1)]]

Caused by op 'gradients/sampled_softmax_loss_2/Mul_grad/BroadcastGradientArgs', defined at:
  File "D:\python3_5anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\python3_5anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\python3_5anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\python3_5anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\python3_5anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File